In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
# Figure out how to import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np 

In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2020-05-15 22:17:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.31.128, 2607:f8b0:400c:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-05-15 22:17:07 (129 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [20]:
from tensorflow.keras import regularizers
import tensorflow as tf

model = Sequential()
model.add(tf.keras.layers.Embedding(total_words, 64, input_length=max_sequence_len-1)) #-1 since last one is target
model.add(tf.keras.layers.Bidirectional(LSTM(20, return_sequences=True)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(LSTM(20)))
model.add(tf.keras.layers.Dense(total_words, kernel_regularizer = regularizers.l2(0.01), activation="softmax"))
# Pick an optimizer
model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics=["categorical_crossentropy", "accuracy"])
print(model.summary())


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 10, 64)            205504    
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 10, 40)            13600     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 40)            0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 40)                9760      
_________________________________________________________________
dense_9 (Dense)              (None, 3211)              131651    
Total params: 360,515
Trainable params: 360,515
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 5s 11ms/step - loss: 7.1657 - categorical_crossentropy: 6.9741 - accuracy: 0.0206
Epoch 2/100
484/484 [==============================] - 5s 11ms/step - loss: 6.9123 - categorical_crossentropy: 6.6928 - accuracy: 0.0208
Epoch 3/100
484/484 [==============================] - 5s 11ms/step - loss: 6.8845 - categorical_crossentropy: 6.6688 - accuracy: 0.0215
Epoch 4/100
484/484 [==============================] - 5s 11ms/step - loss: 6.8238 - categorical_crossentropy: 6.6016 - accuracy: 0.0233
Epoch 5/100
484/484 [==============================] - 6s 12ms/step - loss: 6.7878 - categorical_crossentropy: 6.5657 - accuracy: 0.0263
Epoch 6/100
484/484 [==============================] - 6s 11ms/step - loss: 6.7656 - categorical_crossentropy: 6.5468 - accuracy: 0.0277
Epoch 7/100
442/484 [==========================>...] - ETA: 0s - loss: 6.7359 - categorical_crossentropy: 6.5206 - accuracy: 0.0289

In [0]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)